### Create neural network for training Variational autoencoder (VAE). (Use MNIST Dataset)

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

### **Load Dateset**

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

full_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)


train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False) 

### **Model**

In [3]:
class ConvolutionalVAE(nn.Module):
    def __init__(self, latent_dim):
        super(ConvolutionalVAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.flatten = nn.Flatten() # Flatten the output of the convolutional encoder
        self.fc_mu = nn.Linear(32 * 7 * 7, latent_dim)  # Adjust the input size based on your image size and encoder layers
        self.fc_logvar = nn.Linear(32 * 7 * 7, latent_dim)
        self.unflatten = nn.Unflatten(1, (32, 7, 7)) # Unflatten before the decoder
        self.linear_decoder = nn.Linear(latent_dim, 32 * 7 * 7) 
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )
        

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        hidden = self.encoder(x)
        hidden = self.flatten(hidden) # Flatten for the fully connected layers
        mu = self.fc_mu(hidden)
        logvar = self.fc_logvar(hidden)
        z = self.reparameterize(mu, logvar)
        z = self.linear_decoder(z)
        z = self.unflatten(z) # Unflatten before the convolutional decoder
        decoded = self.decoder(z)
        return decoded, mu, logvar


def loss_function(reconstructed_x, x, mu, logvar):
    reconstruction_loss = nn.MSELoss()(reconstructed_x, x)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return reconstruction_loss + kl_divergence



### **Training Loop**

In [4]:
vae_model = ConvolutionalVAE(latent_dim = 64)
optimizer = torch.optim.Adam(vae_model.parameters(), lr=0.001)

epoch = 1
for i in range(epoch):
    ite = 0
    for x,y in train_loader:
        
        optimizer.zero_grad()
        
        outputs, mu, logvar  = vae_model(x)
        loss = loss_function(outputs, x, mu, logvar)
        
        loss.backward()
        optimizer.step()
        
        ite += 1
        
        if ite % 10 == 0:
            print("iteration: ", ite , "Loss: ", float(loss))

iteration:  10 Loss:  3.396404981613159
iteration:  20 Loss:  2.2395801544189453
iteration:  30 Loss:  1.9924795627593994
iteration:  40 Loss:  1.8015836477279663
iteration:  50 Loss:  1.6671171188354492
iteration:  60 Loss:  1.5486398935317993
iteration:  70 Loss:  1.4438987970352173
iteration:  80 Loss:  1.3507699966430664
iteration:  90 Loss:  1.25626540184021
iteration:  100 Loss:  1.190504550933838
iteration:  110 Loss:  1.1203495264053345
iteration:  120 Loss:  1.0626089572906494
iteration:  130 Loss:  1.0215226411819458
iteration:  140 Loss:  1.0031187534332275
iteration:  150 Loss:  0.9825464487075806
iteration:  160 Loss:  0.9664984941482544
iteration:  170 Loss:  0.9611880779266357
iteration:  180 Loss:  0.9522223472595215
iteration:  190 Loss:  0.9502744078636169
iteration:  200 Loss:  0.9416422843933105
iteration:  210 Loss:  0.9427673816680908
iteration:  220 Loss:  0.9351805448532104
iteration:  230 Loss:  0.9383175373077393
iteration:  240 Loss:  0.9350405335426331
itera

### **Evaluation**

In [5]:
vae_model.eval

with torch.no_grad():
    mu_list = []
    logvar_list = []
    for x,y in test_loader:
        
        _, mu, logvar = vae_model(x)
        mu_list.append(mu)
        logvar_list.append(logvar)

logvar_list = torch.stack(logvar_list)     
mu_list = torch.stack(mu_list)
print(mu_list.shape)
mu_list

torch.Size([120, 100, 64])


tensor([[[ 8.5118e-06,  2.0026e-05,  2.0080e-05,  ..., -1.1572e-05,
          -4.3315e-06,  1.0980e-05],
         [ 8.5118e-06,  2.0026e-05,  2.0080e-05,  ..., -1.1572e-05,
          -4.3315e-06,  1.0980e-05],
         [-1.8285e-04, -4.5074e-05, -2.2319e-04,  ...,  5.8072e-05,
           7.6843e-05,  2.1030e-04],
         ...,
         [-7.7269e-05, -8.1903e-05,  8.6583e-05,  ..., -1.8766e-04,
           5.0944e-05, -7.6514e-05],
         [ 8.5118e-06,  2.0026e-05,  2.0080e-05,  ..., -1.1572e-05,
          -4.3315e-06,  1.0980e-05],
         [-4.9843e-04,  3.1446e-04, -8.0263e-05,  ..., -1.5430e-04,
           3.2663e-04, -6.1707e-04]],

        [[ 8.5118e-06,  2.0026e-05,  2.0080e-05,  ..., -1.1572e-05,
          -4.3315e-06,  1.0980e-05],
         [ 8.5118e-06,  2.0026e-05,  2.0080e-05,  ..., -1.1572e-05,
          -4.3315e-06,  1.0980e-05],
         [ 8.5118e-06,  2.0026e-05,  2.0080e-05,  ..., -1.1572e-05,
          -4.3315e-06,  1.0980e-05],
         ...,
         [ 8.5118e-06,  2